In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def scrape_fubon_broker_data(stock_id, start_date, end_date):
    """
    爬取富邦證券卷商買賣超資料
    
    Parameters:
    stock_id (str): 股票代碼
    start_date (str): 開始日期 (格式: YYYY-MM-DD)
    end_date (str): 結束日期 (格式: YYYY-MM-DD)
    
    Returns:
    pandas.DataFrame: 包含卷商買賣超資料的DataFrame
    """
    
    # 構建URL
    url = f"https://fubon-ebrokerdj.fbs.com.tw/z/zg/zgb/zgb0.djhtm"
    params = {
        'a': stock_id,
        'b': stock_id,
        'c': 'E',
        'e': start_date,
        'f': end_date
    }
    
    # 設定headers模擬瀏覽器
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        # 發送請求
        response = requests.get(url, params=params, headers=headers)
        response.encoding = 'big5'  # 設定編碼為big5
        
        # 解析HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 找到資料表格
        data = []
        for table in soup.find_all('table', {'class': 't0'}):
            if not table:
                print("找不到資料表格")
                return None

            # 解析表格資料
            rows = table.find_all('tr')[2:]  # 跳過標題行
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 4:  # 確保欄位數量正確
                    broker = cols[0].text.strip()
                    buy = int(cols[1].text.strip().replace(',', ''))
                    sell = int(cols[2].text.strip().replace(',', ''))
                    net = int(cols[3].text.strip().replace(',', ''))

                    data.append({
                        '券商名稱': broker,
                        '買進張數': buy,
                        '賣出張數': sell,
                        '買賣超張數': net
                    })
        
        # 轉換成DataFrame
        df = pd.DataFrame(data)
        
        
        return df
        
    except Exception as e:
        print(f"發生錯誤: {str(e)}")
        return None

# 使用範例
if __name__ == "__main__":
    dealer_id = "9800"
    dealer_name = "元大"
    sub_dealer_id = "9800"
    sub_dealer_name = "元大證卷"
    start_date = "2025-1-21"
    end_date = "2025-1-21"
    
    df = scrape_fubon_broker_data(stock_id, start_date, end_date)
    if df is not None:
        print(df)
        # 可以選擇將結果保存到CSV
        df.to_csv(f'F:\\文義\\股票資料\\股票歷史資料\\卷商分點資料\\broker_data_{dealer_id}_{dealer_name}_{sub_dealer_id}_{sub_dealer_name}_{start_date}.csv', encoding='utf-8-sig', index=False)

                券商名稱   買進張數  賣出張數  買賣超張數
0      00929復華台灣科技優息  10404   562   9842
1   00937B群益ESG投等債20  12591  3905   8686
2    00665L富邦�琤肭磪囓�2  10767  6505   4262
3                      3583   268   3315
4     00753L中信中國50正2   6084  3038   3046
..               ...    ...   ...    ...
95     00858永豐美國500大      2   448   -446
96                      507   938   -431
97    00915凱基優選高股息30    944  1370   -426
98     00926凱基全球菁英55    179   588   -409
99    00900富邦特選高股息30    241   628   -387

[100 rows x 4 columns]
